In [ ]:
import keras
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
import os.path
import csv
from keras.preprocessing.image import DirectoryIterator, ImageDataGenerator

dataset = 'faces' # one of 'faces' 'chairs' 'celeba'
z_dims = 32
beta = 0.1

if dataset == 'faces':
    channels = 1
    likelihood = 'bernoulli'
elif dataset == 'chairs':
    channels = 1
    likelihood = 'bernoulli'
elif dataset == 'celeba':
    channels = 3
    likelihood = 'gaussian'

grayscale = dataset == 'faces' or dataset == 'chairs'

In [ ]:
if dataset == 'faces':
    data = np.load('faces-labelled.npz')['images']
elif dataset == 'chairs':
    data = np.load('{}.npy'.format(dataset))
elif dataset == 'celeba':
    dir_iter = DirectoryIterator(
        directory='/tmp/tmp',
        image_data_generator=ImageDataGenerator(
            #validation_split=
            rescale=1/255,
        ),
        target_size=(64, 64),
        color_mode='rgb', # grayscale
        class_mode='input',
        batch_size=1,
        shuffle=False,
        #seed= for shuffling
        #subset='training'
        interpolation='bilinear',
    )

In [ ]:
class NormalSampler(keras.layers.Layer):
    def __init__(self):
        super(NormalSampler, self).__init__()

    def call(self, mu_logvar):
        mu, logvar = mu_logvar
        epsilon = K.random_normal(shape=K.shape(mu))
        std = K.exp(logvar / 2)
        return mu + epsilon*std


inputs = keras.Input(shape=(channels, 64, 64))

# encoder
x = keras.layers.Conv2D(filters=32, kernel_size=4, strides=2, padding='same', activation='relu')(inputs)
x = keras.layers.Conv2D(filters=32, kernel_size=4, strides=2, padding='same', activation='relu')(x)
x = keras.layers.Conv2D(filters=64, kernel_size=4, strides=2, padding='same', activation='relu')(x)
x = keras.layers.Conv2D(filters=64, kernel_size=4, strides=2, padding='same', activation='relu')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(units=256, activation='relu')(x)
mu = keras.layers.Dense(units=z_dims, activation=None)(x)
logvar = keras.layers.Dense(units=z_dims, activation=None)(x)
z = NormalSampler()([mu, logvar])

encoder_mu = keras.Model(inputs=inputs, outputs=mu, name='encoder_mu')
encoder = keras.Model(inputs=inputs, outputs=[z, mu, logvar], name='encoder')


# decoder
d_inputs = keras.Input(shape=(z_dims,))
x = keras.layers.Dense(units=256, activation='relu')(d_inputs)
x = keras.layers.Dense(units=64*4*4, activation='relu')(x)
x = keras.layers.Reshape((64, 4, 4))(x)
x = keras.layers.Conv2DTranspose(filters=64, kernel_size=4, strides=2, padding='same', activation='relu')(x)
x = keras.layers.Conv2DTranspose(filters=32, kernel_size=4, strides=2, padding='same', activation='relu')(x)
x = keras.layers.Conv2DTranspose(filters=32, kernel_size=4, strides=2, padding='same', activation='relu')(x)
x = keras.layers.Conv2DTranspose(filters=channels, kernel_size=4, strides=2, padding='same', activation=None)(x)
decoder = keras.Model(inputs=d_inputs, outputs=x, name='decoder')


# combined
outputs = decoder(encoder(inputs)[0])
model = keras.Model(inputs=inputs, outputs=outputs)


model_path = os.path.join('checkpoints', '{}-{}-{}'.format(dataset, z_dims, beta))
log_path = os.path.join('checkpoints', '{}-{}-{}.csv'.format(dataset, z_dims, beta))
if os.path.exists(model_path):
    model.load_weights(model_path)
    print('Loaded existing checkpoint')

In [ ]:
# draw full traversal of all 32 latents

#i = 30100 # faces
#i = 15000 # faces
i = 37000 # faces
#i = 500 # chairs
#i = 70 # chairs

if dataset == 'chairs' or dataset == 'faces':
    test = data[i:i+1]
elif dataset == 'celeba':
    test = dir_iter[i][0]

fig, ax = plt.subplots()
if grayscale:
    test2 = 1/(1+np.exp(-test))
    ax.imshow(np.reshape(test2, (64, 64)), cmap='gray')
else:
    test2 = test.reshape(3, 64, 64)
    test2 = np.transpose(test2, (1, 2, 0))
    ax.imshow(test2)

    
z = encoder_mu.predict(test)

z_indices = range(32)
n = len(z_indices)

#z_range = [-1, -0.5, 0, 0.5, 1]
z_range = [-3, -2, -1, 0, 1, 2, 3]
m = len(z_range)

fig, axs = plt.subplots(n, m, figsize=(2*m,2*n))
fig.subplots_adjust(wspace=0, hspace=0)

for i, z_i in enumerate(z_indices):
    z_copy = np.copy(z)
    for j, v in enumerate(z_range):
        z_copy[0,z_i] = v
        recon = decoder.predict(z_copy)
        
        recon = 1/(1+np.exp(-recon))
        if grayscale:
            axs[i, j].imshow(np.reshape(recon, (64, 64)), cmap='gray', aspect='auto')
        else:
            recon = recon.reshape(3, 64, 64)
            recon = np.transpose(recon, (1, 2, 0))
            axs[i, j].imshow(recon)
        axs[i, j].set_axis_off()
        #axs[i, j].set_title(str(i))


In [ ]:
# traversal of a single latent over various images

#images = [130, 500, 66, 3000] # chairs
images = [3000, 37000, 40100, 55000, 70000] # faces
n = len(images)

z_i = 21

#z_range = [-1, -0.5, 0, 0.5, 1]
z_range = [-3, -1.5, 0, 1.5, 3]
m = len(z_range)

fig, axs = plt.subplots(n, m, figsize=(2*m,2*n))
fig.subplots_adjust(wspace=0, hspace=0)

for i, image_i in enumerate(images):
    test = data[image_i:image_i+1]
    z = encoder_mu.predict(test)
    z_copy = np.copy(z)
    for j, v in enumerate(z_range):
        z_copy[0,z_i] = v
        recon = decoder.predict(z_copy)
        recon = 1/(1+np.exp(-recon))
        
        axs[i, j].imshow(np.reshape(recon, (64, 64)), cmap='gray', aspect='auto')
        axs[i, j].set_axis_off()